In [198]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame
import random as rd  
from math import sqrt

In [199]:
dataTable = pd.read_csv('iris.csv', names =['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'flower']) 

In [200]:
xMatrix = np.array(dataTable[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']])
yVector = np.array(dataTable['flower'])

In [201]:
dataTable

,sepal_length,sepal_width,petal_length,petal_width,flower
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
95,5.7,3.0,4.2,1.2,1
96,5.7,2.9,4.2,1.3,1
97,6.2,2.9,4.3,1.3,1
98,5.1,2.5,3.0,1.1,1


In [202]:
def crossValidationSplit(xMatrix, yVector, n_folds): 
    xMatrixSplit = []
    yVectorSplit = []
    xMatrixCopy = xMatrix

    yVectorCopy = yVector
    foldSize = int(len(xMatrix) / n_folds) 
    
    for i in range(n_folds): 
        foldXM = [] 
        foldYV = []
        while len(foldXM) < foldSize: 
            index = rd.randrange(len(xMatrixCopy)) 
            foldXM.append(xMatrixCopy[index]) 
            xMatrixCopy = np.delete(xMatrixCopy, index, axis=0)
            foldYV.append(yVectorCopy[index]) 
            yVectorCopy = np.delete(yVectorCopy, index)
        xMatrixSplit.append(foldXM)
        yVectorSplit.append(foldYV)
    return xMatrixSplit, yVectorSplit

def kFoldCrossValidation(xMatrix, yVector, kParts, algorithm, *args):
    xMFolds, yVFolds = crossValidationSplit(xMatrix, yVector, kParts) 
    scores = list() 
    count = 0;
    for fold in xMFolds: 
        xTrain = xMFolds
        yTrain = yVFolds
        
        xTest = xTrain[count]
        yTest = np.array(yTrain[count])
        
        np.delete(xTrain,count)
        np.delete(yTrain,count)
        
        yTrain = np.stack(yTrain)
        yTrain = yTrain.ravel()
        xTrainAux = []

        for i in range(kParts):
            for j in range(len(xTrain[i])):
                xTrainAux.append(xTrain[i][j])
        xTrain = np.stack(xTrainAux) 
        yTest = np.where(yTest<0, 0, yTest)
        predicted = algorithm(xTrain, yTrain, xTest, *args)
        for i in range(len(predicted)):
            print("Predicted: ",predicted[i],"Actual: ",yTest[i])
        accuracy = np.array([x - y for x, y in zip(predicted, yTest)])/kParts
        scores.append(accuracy)  
    return scores

In [203]:
def compute_Gaussians(xTrain,yTrain):
    means = list()
    variances = list()
    prob = list()
    
    for i in range(np.amax(yTrain)+1):
        means.append(np.mean(xTrain[np.argwhere(yTrain == i)] , 0 ))
        variances.append(np.std(xTrain[np.argwhere(yTrain == i)] , 0 )) 
        prob.append(len(np.argwhere(yTrain ==i)) /len(yTrain))
 
    return prob, means, variances

In [228]:
def NBayes_Gaussian(xTrain, yTrain, xTest): 
    prob, means, variances = compute_Gaussians(xTrain,yTrain)
    prediction = list() 
    APLog = np.zeros([len(xTest),len(prob) ])   
    for i in range(len(xTest)):
        for j in range(len(prob)): 
            APLog[i,j] = prob[j] - 1/2*np.sum(np.log(2*np.pi*variances[j]))-1/2*np.sum(np.divide(np.square(xTest[i]-means[j]),variances[j]))
         
 
    prediction = np.argmax(APLog,1)  
     
            
    return prediction
        

In [229]:
kParts = 5
kFoldCrossValidation(xMatrix, yVector, kParts, NBayes_Gaussian)


Predicted:  0 Actual:  0
Predicted:  1 Actual:  1
Predicted:  1 Actual:  1
Predicted:  1 Actual:  1
Predicted:  0 Actual:  0
Predicted:  1 Actual:  1
Predicted:  0 Actual:  0
Predicted:  0 Actual:  0
Predicted:  1 Actual:  1
Predicted:  0 Actual:  0
Predicted:  1 Actual:  1
Predicted:  1 Actual:  1
Predicted:  0 Actual:  0
Predicted:  1 Actual:  1
Predicted:  0 Actual:  0
Predicted:  1 Actual:  1
Predicted:  0 Actual:  0
Predicted:  0 Actual:  0
Predicted:  0 Actual:  0
Predicted:  1 Actual:  1
Predicted:  0 Actual:  0
Predicted:  1 Actual:  1
Predicted:  1 Actual:  1
Predicted:  1 Actual:  1
Predicted:  0 Actual:  0
Predicted:  1 Actual:  1
Predicted:  0 Actual:  0
Predicted:  0 Actual:  0
Predicted:  1 Actual:  1
Predicted:  0 Actual:  0
Predicted:  1 Actual:  1
Predicted:  1 Actual:  1
Predicted:  0 Actual:  0
Predicted:  1 Actual:  1
Predicted:  0 Actual:  0
Predicted:  1 Actual:  1
Predicted:  0 Actual:  0
Predicted:  0 Actual:  0
Predicted:  0 Actual:  0
Predicted:  1 Actual:  1


[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.])]